In [ ]:
cd /content/drive/MyDrive/Hackathon

[Errno 2] No such file or directory: '/content/drive/MyDrive/Hackathon'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_excel('tamil_nadu_warehouses.xlsx')
dictionary = df.to_dict(orient='records')
tamil_nadu_warehouses = {item['Unnamed: 0']: {key: value for key, value in item.items() if key != 'Unnamed: 0'} for item in dictionary}
print(tamil_nadu_warehouses)


{'Warehouse_A': {'location': 'Chennai', 'soil_type': 'Red Loam', 'land_area_sqkm': 120, 'district': 'Chennai'}, 'Warehouse_B': {'location': 'Coimbatore', 'soil_type': 'Alluvial Soil', 'land_area_sqkm': 150, 'district': 'Coimbatore'}, 'Warehouse_C': {'location': 'Madurai', 'soil_type': 'Black Soil', 'land_area_sqkm': 100, 'district': 'Madurai'}, 'Warehouse_D': {'location': 'Trichy', 'soil_type': 'Red Sandy Soil', 'land_area_sqkm': 80, 'district': 'Trichy'}, 'Warehouse_E': {'location': 'Salem', 'soil_type': 'Laterite Soil', 'land_area_sqkm': 110, 'district': 'Salem'}}


In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
soil_suitability_tn = {
    'Wheat': {'Red Loam': 1.2, 'Alluvial Soil': 1.0, 'Black Soil': 1.5, 'Red Sandy Soil': 0.8, 'Laterite Soil': 0.9},
    'Rice': {'Red Loam': 1.3, 'Alluvial Soil': 1.1, 'Black Soil': 1.4, 'Red Sandy Soil': 0.7, 'Laterite Soil': 0.8},
   'Maize': {'Red Loam': 1.0, 'Alluvial Soil': 1.3, 'Black Soil': 1.2, 'Red Sandy Soil': 0.9, 'Laterite Soil': 1.1},
}

weights = {'soil_suitability': 0.6, 'land_area': 0.4}
data = []

for crop,suitability in soil_suitability_tn.items():
    for warehouse, attributes in tamil_nadu_warehouses.items():
        score = suitability[attributes['soil_type']]
        land_area = attributes['land_area_sqkm']
        allocation = score * weights['soil_suitability'] + land_area * weights['land_area']
        data.append([warehouse,attributes['location'],attributes['district'],crop,attributes['soil_type'],score,land_area,allocation])

df = pd.DataFrame(data, columns=['warehouse', 'location', 'district', 'crop', 'soil_type', 'soil_score', 'land_area', 'allocation'])
print(df)

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[['crop', 'soil_type']])
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['crop', 'soil_type']))

X = pd.concat([df[['soil_score', 'land_area']], encoded_features_df], axis=1)
y = df['allocation']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

import pandas as pd

Input_data = pd.read_excel('Input_data.xlsx')
print(Input_data)

Input_data['soil_score'] = Input_data.apply(lambda row: soil_suitability_tn[row['crop']][row['soil_type']], axis=1)

encoded_features = encoder.transform(Input_data[['crop', 'soil_type']])
encoded_data = pd.concat([Input_data[['soil_score', 'land_area']], pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['crop', 'soil_type']))], axis=1)

predictions = model.predict(encoded_data)
Input_data['predicted_allocation'] = predictions
Input_data['normalized_allocation'] = Input_dat\ .groupby('crop')['predicted_allocation'].transform(lambda x: x / x.sum())

crop_quantities = {'Rice': 10000,'Maize': 7600,'Wheat': 8000}

Input_data = Input_data[Input_data['crop'].isin(crop_quantities.keys())]
Input_data['allocated_quantity'] = Input_data.apply(lambda row: row['normalized_allocation'] * crop_quantities[row['crop']], axis=1)

print("\nPredicted Allocations and Allocated Quantities:")
print(Input_data[['crop', 'soil_type', 'predicted_allocation', 'normalized_allocation', 'allocated_quantity']])

Input_data=Input_data.drop(['normalized_allocation','predicted_allocation'],axis=1)

print(Input_data)

Input_data.to_excel('allocation.xlsx', index=False)




      warehouse    location    district   crop       soil_type  soil_score  \
0   Warehouse_A     Chennai     Chennai  Wheat        Red Loam         1.2   
1   Warehouse_B  Coimbatore  Coimbatore  Wheat   Alluvial Soil         1.0   
2   Warehouse_C     Madurai     Madurai  Wheat      Black Soil         1.5   
3   Warehouse_D      Trichy      Trichy  Wheat  Red Sandy Soil         0.8   
4   Warehouse_E       Salem       Salem  Wheat   Laterite Soil         0.9   
5   Warehouse_A     Chennai     Chennai   Rice        Red Loam         1.3   
6   Warehouse_B  Coimbatore  Coimbatore   Rice   Alluvial Soil         1.1   
7   Warehouse_C     Madurai     Madurai   Rice      Black Soil         1.4   
8   Warehouse_D      Trichy      Trichy   Rice  Red Sandy Soil         0.7   
9   Warehouse_E       Salem       Salem   Rice   Laterite Soil         0.8   
10  Warehouse_A     Chennai     Chennai  Maize        Red Loam         1.0   
11  Warehouse_B  Coimbatore  Coimbatore  Maize   Alluvial Soil  

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

farmer_data = pd.read_excel('Farmers.xlsx')
print(farmer_data)


   Farmer ID Farmer Name       Soil Type  Land Area (sq km)         Location
0      A1001    Farmer_1  Red Sandy Soil               4.06       Coimbatore
1      A1002    Farmer_2      Black Soil               7.45          Chennai
2      A1003    Farmer_3   Alluvial Soil               1.98  Tiruchirappalli
3      A1004    Farmer_4   Laterite Soil               1.05      Tirunelveli
4      A1005    Farmer_5   Alluvial Soil               6.21      Tirunelveli
..       ...         ...             ...                ...              ...
95     E1096   Farmer_96  Red Sandy Soil               1.38       Coimbatore
96     E1097   Farmer_97   Alluvial Soil               9.05          Vellore
97     E1098   Farmer_98        Red Loam               3.72       Coimbatore
98     E1099   Farmer_99        Red Loam               7.40          Madurai
99     E1100  Farmer_100      Black Soil               8.93          Vellore

[100 rows x 5 columns]


In [ ]:
warehouse_data = pd.read_excel('allocation.xlsx')
warehouse_data = warehouse_data.to_dict(orient='records')

warehouse_dict = {}
for entry in warehouse_data:
    warehouse_name = entry["warehouse"]
    crop = entry["crop"]
    allocated_quantity = entry["allocated_quantity"]

    if warehouse_name not in warehouse_dict:
        warehouse_dict[warehouse_name] = {}

    if crop in warehouse_dict[warehouse_name]:
        warehouse_dict[warehouse_name][crop] += allocated_quantity
    else:
        warehouse_dict[warehouse_name][crop] = allocated_quantity

print(warehouse_dict)


{'Warehouse_A': {'Rice': 3467.716574668884}, 'Warehouse_B': {'Wheat': 8000.0}, 'Warehouse_C': {'Maize': 7600.0}, 'Warehouse_D': {'Rice': 2752.615809440613}, 'Warehouse_E': {'Rice': 3779.667615890503}}
